<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2024-01-16 21:26:13--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  13.7MB/s    in 17s     

2024-01-16 21:26:31 (14.5 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip -q concrete_data_week4.zip

In [ ]:
pwd

'/content'

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [ ]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [ ]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [ ]:
model1 = Sequential()

In [ ]:
model1.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [ ]:
model1.add(Dense(num_classes, activation='softmax'))

In [ ]:
model1.layers

In [ ]:
model1.layers[0].layers

In [ ]:
model1.layers[0].trainable = False

In [ ]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [ ]:
fit_history = model1.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

<ipython-input-42-52ebed9b3d5d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = model1.fit_generator(


Epoch 1/2
301/301 [==============================] - 186s 614ms/step - loss: 0.1380 - accuracy: 0.9545 - val_loss: 0.0279 - val_accuracy: 0.9948
Epoch 2/2
301/301 [==============================] - 182s 603ms/step - loss: 0.0214 - accuracy: 0.9954 - val_loss: 0.0164 - val_accuracy: 0.9966


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
from tensorflow.keras.models import load_model

model2 = load_model('classifier_resnet_model.h5')

In [ ]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.


In [ ]:
evaluation_vgg = model1.evaluate_generator(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size,
    verbose=1
)

<ipython-input-74-7cab6b038acf>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation_vgg = model1.evaluate_generator(


15/15 [==============================] - 2s 137ms/step - loss: 0.0157 - accuracy: 0.9979


In [ ]:
evaluation_resnet = model2.evaluate_generator(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size,
    verbose=1
)

<ipython-input-75-ae8a64d75017>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluation_resnet = model2.evaluate_generator(


15/15 [==============================] - 2s 111ms/step - loss: 0.0029 - accuracy: 1.0000


In [ ]:
print(f"The performance of the classifier on the test set using vgg is: {evaluation_vgg[1]}")
print(f"The performance of the classifier on the test set using resnet is: {evaluation_resnet[1]}")

The performance of the classifier on the test set using vgg is: 0.9979166388511658
The performance of the classifier on the test set using resnet is: 1.0


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
prediction_vgg = model1.predict_generator(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size,
    verbose=1
)

<ipython-input-76-076319d41fc2>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction_vgg = model1.predict_generator(


15/15 [==============================] - 2s 134ms/step


In [ ]:
prediction_resnet =  model1.predict_generator(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size,
    verbose=1
)

<ipython-input-77-591626a77565>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction_resnet =  model1.predict_generator(


15/15 [==============================] - 2s 131ms/step


In [ ]:
dict_ = {0:"Negative" , 1:"Positive"}
resnet_pred = np.argmax(prediction_resnet[:5] , axis = 1)
vgg_pred = np.argmax(prediction_vgg[:5] , axis = 1)

print(f"The predictions of the first five images in the test set using the classifier based on vgg are:")
for i in range(5):
    print(dict_[vgg_pred[i]])
print("-------------------------------------------------------------------------------------------------------------")
print(f"The predictions of the first five images in the test set using the classifier based on resnet are:")
for i in range(5):
    print(dict_[resnet_pred[i]])

The predictions of the first five images in the test set using the classifier based on vgg are:
Negative
Negative
Negative
Negative
Negative
-------------------------------------------------------------------------------------------------------------
The predictions of the first five images in the test set using the classifier based on resnet are:
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).